# Uniview module for a type Ia SN 

*Aaron Geller, 2019*

I'm following [this awesome tutorial](https://www.clicktorelease.com/blog/vertex-displacement-noise-3d-webgl-glsl-three-js/) to create the X-ray portion.  Other bits of this were drawn from my [SN1987a module](https://github.com/ageller/UniviewSN1987a).

The X-rays will look close to Tycho's SN.

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [2]:
Template = """mesh
{  
    data center    ./modules/$folderName/center.raw
    data  sphere ./modules/$folderName/sphere400.obj
    data  quad ./modules/$folderName/quad.3ds
    
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f SNRadScale 1000 | public | desc "max SN radius" | widget slider | range 0 1e4
            vec1f SNAlpha 1 | public | desc "SN alpha " | widget slider | range 0 1
            
            #yellow
            vec1f XrayRadMax1 26 | public | desc "max Xray radius" | widget slider | range 0 500
            vec1f XrayTimeMax1 400 | public | desc "max Xray time" | widget slider | range 0 200
            vec1f XrayTimeMin1 50.15 | public | desc "min  Xray time" | widget slider | range 0 200
            vec1f XrayAlpha1 1. | public | desc "Xray alpha " | widget slider | range 0 1
            vec1f XrayAlphaPow1 4. | public | desc "Xray alpha " | widget slider | range 0 20
            vec1f XrayNoiseSpeed1 0.004 | public | desc "Xray noise speed " | widget slider | range 0 0.1
            vec1f XrayNoisePulseHeight1 0 | public | desc "Xray noise pulse height " | widget slider | range 0 2
            vec1f XrayNoiseDisplacementHeight1 0.2 | public | desc "Xray noise displacement height " | widget slider | range 0 3
            vec1f XrayNoiseTurbulenceDetail1 5 | public | desc "Xray noise speed " | widget slider | range 0 10
            
            #orange
            vec1f XrayRadMax2 25 | public | desc "max Xray radius" | widget slider | range 0 500
            vec1f XrayTimeMax2 400 | public | desc "max Xray time" | widget slider | range 0 200
            vec1f XrayTimeMin2 50 | public | desc "min  Xray time" | widget slider | range 0 200
            vec1f XrayAlpha2 1. | public | desc "Xray alpha " | widget slider | range 0 1
            vec1f XrayAlphaPow2 4. | public | desc "Xray alpha " | widget slider | range 0 20
            vec1f XrayNoiseSpeed2 0.002 | public | desc "Xray noise speed " | widget slider | range 0 0.1
            vec1f XrayNoisePulseHeight2 0 | public | desc "Xray noise pulse height " | widget slider | range 0 2
            vec1f XrayNoiseDisplacementHeight2 0.15 | public | desc "Xray noise displacement height " | widget slider | range 0 3
            vec1f XrayNoiseTurbulenceDetail2 1 | public | desc "Xray noise speed " | widget slider | range 0 10
             
            #purple
            vec1f XrayRadMax3 27 | public | desc "max Xray radius" | widget slider | range 0 500
            vec1f XrayTimeMax3 400 | public | desc "max Xray time" | widget slider | range 0 200
            vec1f XrayTimeMin3 45 | public | desc "min  Xray time" | widget slider | range 0 200
            vec1f XrayAlpha3 0.3 | public | desc "Xray alpha " | widget slider | range 0 1
            vec1f XrayAlphaPow3 4. | public | desc "Xray alpha " | widget slider | range 0 20
            vec1f XrayNoiseSpeed3 0.001 | public | desc "Xray noise speed " | widget slider | range 0 0.1
            vec1f XrayNoisePulseHeight3 0 | public | desc "Xray noise pulse height " | widget slider | range 0 2
            vec1f XrayNoiseDisplacementHeight3 0.2 | public | desc "Xray noise displacement height " | widget slider | range 0 3
            vec1f XrayNoiseTurbulenceDetail3 0.25 | public | desc "Xray noise speed " | widget slider | range 0 10
             

            vec1f coronaRad 1  | public | desc "size of stellar corona" | widget slider | range 0 50
            vec1f coronaTeff 16000 | public | desc "Teff of stellar corona for color" | widget slider | range 0 19000
            vec1f coronaEndTime 3 | public | desc "time to stop showing star" 
            
            vec1f eventTime 400 | public | desc "event time " #| widget slider | range 0 30
            vec1f transitionLength 5 | public | desc "transition length in seconds" 
            bool jump true | public  | desc "jump to time without transition"          
            
        }
    }
    
    ############# to hold the time information
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/pass0.vs
                fragmentShader ./modules/$folderName/state.fs
                textureFBO stateTexture stateTexture
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                parameter2f timeRange 0 400
           }
        }      
    }
    
    
    ############# Exanding X-ray sphere yellow
    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader ./modules/$folderName/xray.vs
                fragmentShader   ./modules/$folderName/xray.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmapSummer.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                
                stateManagerVar __objectName__.XrayRadMax1  radMax
                stateManagerVar __objectName__.XrayTimeMax1  timeMax
                stateManagerVar __objectName__.XrayTimeMin1  timeMin
                stateManagerVar __objectName__.XrayAlpha1  alpha
                stateManagerVar __objectName__.XrayAlphaPow1  alphaPow
                stateManagerVar __objectName__.XrayNoiseSpeed1  noiseSpeed
                stateManagerVar __objectName__.XrayNoisePulseHeight1  noisePulseHeight
                stateManagerVar __objectName__.XrayNoiseDisplacementHeight1  noiseDisplacementHeight
                stateManagerVar __objectName__.XrayNoiseTurbulenceDetail1  noiseTurbulenceDetail
                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
    ############# Exanding X-ray sphere orange
    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader ./modules/$folderName/xray.vs
                fragmentShader   ./modules/$folderName/xray.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmapYlOrBr.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                
                stateManagerVar __objectName__.XrayRadMax2  radMax
                stateManagerVar __objectName__.XrayTimeMax2  timeMax
                stateManagerVar __objectName__.XrayTimeMin2  timeMin
                stateManagerVar __objectName__.XrayAlpha2  alpha
                stateManagerVar __objectName__.XrayAlphaPow2  alphaPow
                stateManagerVar __objectName__.XrayNoiseSpeed2  noiseSpeed
                stateManagerVar __objectName__.XrayNoisePulseHeight2  noisePulseHeight
                stateManagerVar __objectName__.XrayNoiseDisplacementHeight2  noiseDisplacementHeight
                stateManagerVar __objectName__.XrayNoiseTurbulenceDetail2  noiseTurbulenceDetail
                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
        
    ############# Exanding X-ray sphere purple
    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader ./modules/$folderName/xray.vs
                fragmentShader   ./modules/$folderName/xray.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmapPurples.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                
                stateManagerVar __objectName__.XrayRadMax3  radMax
                stateManagerVar __objectName__.XrayTimeMax3  timeMax
                stateManagerVar __objectName__.XrayTimeMin3  timeMin
                stateManagerVar __objectName__.XrayAlpha3  alpha
                stateManagerVar __objectName__.XrayAlphaPow3  alphaPow
                stateManagerVar __objectName__.XrayNoiseSpeed3  noiseSpeed
                stateManagerVar __objectName__.XrayNoisePulseHeight3  noisePulseHeight
                stateManagerVar __objectName__.XrayNoiseDisplacementHeight3  noiseDisplacementHeight
                stateManagerVar __objectName__.XrayNoiseTurbulenceDetail3  noiseTurbulenceDetail
                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
    ############# Optical explosion (will tie this to the light curve)
    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader ./modules/$folderName/SN.vs
                fragmentShader   ./modules/$folderName/SN.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmapOptical.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }  
                
                stateManagerVar __objectName__.SNRadScale  radScale
                stateManagerVar __objectName__.SNAlpha  SNAlpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }

    ############# stellar corona   
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/corona.gs
                vertexShader ./modules/$folderName/corona.vs
                fragmentShader   ./modules/$folderName/corona.fs
                textureFBO stateTexture stateTexture
                texture bb ./modules/$folderName/bb.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }
                
                stateManagerVar __objectName__.coronaRad  coronaRad
                stateManagerVar __objectName__.coronaTeff  coronaTeff
                stateManagerVar __objectName__.coronaEndTime  coronaEndTime
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }


}"""

### SN class

In [3]:
class SNIa():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("pass0.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("state.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("xray.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("xray.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [4]:
model = SNIa(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [5]:
scene.setname("SNIa")
scene.setparent("Extragalactic")
scene.setunit(1e12)
scene.setentrydist(1e5)
scene.setstaticposition(-2.07892522, 1.65798914, 2.27577283)

model.object.setcameraradius(1)
model.object.setcoord(scene.name)
model.object.setname("SNIa")
model.object.setguiname("/KavliLecture/Zooniverse/SNIa")
model.object.settargetradius(100)
model.object.showatstartup(False)
model.cr = 1000

modinfo.setname("SNIa")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup> and Chris Lintott<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Oxford University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for a SN Ia")
modinfo.setthumbnail("data/SN1aScreenshotThumb.png")
modinfo.setversion("1.0")

generator.generate("SNIa",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [6]:
from config import Settings

In [7]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


## Create colormap texture

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [9]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap, fname):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig(fname, bbox_inches='tight',pad_inches=0)

plot_cmap('plasma', 'data/cmapOptical.png')
plot_cmap('Purples', 'data/cmapPurples.png')
#plot_cmap('Blues', 'data/cmapBlues.png')
#plot_cmap('Greens', 'data/cmapGreens.png')
plot_cmap('YlOrBr', 'data/cmapYlOrBr.png')
#plot_cmap('YlGn', 'data/cmapYlGn.png')
#plot_cmap('YlGn_r', 'data/cmapYlGn_r.png')
plot_cmap('summer', 'data/cmapSummer.png')
#plot_cmap('Reds', 'data/cmapReds.png')


## Position  of SN 2014J in M82

https://en.wikipedia.org/wiki/SN_2014J

In [10]:
from astropy.coordinates import SkyCoord
from astropy import units

In [11]:
RA = "9h 55m 42.217s" #right ascension
Dec= "69d 40m 26.56s" #declination
dist = (3500000 *units.pc).to(units.Mpc) #distance

coord = SkyCoord(RA, Dec, dist)

print(coord.galactic.cartesian)

(-2.07892522, 1.65798914, 2.27577283) Mpc
